In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor

from keras.optimizers import Adam
from keras.models import load_model, Model, Sequential
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Input
from keras.models import Sequential
from keras import regularizers


%matplotlib  inline

C:\Users\EMO\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_pickle("df_t_2_")
# Drop some features that were not usefell for the task
df.drop(columns=["POS_CAT_before","POS_CAT_during","num_trx_during","num_trx_before"], inplace=True)
df_only_pos = df[df["flags"]==1]
y = df_only_pos["sums"]
X = df_only_pos
X.drop(columns=["sums","flags"], inplace=True)


(5000, 10)


C:\Users\EMO\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
#Drop  outliers
print(X.shape[0]-X[y<1000000].shape[0])
X = X[y<1000000]
y = y[y<1000000]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5) 


samples_train = X_train.shape[0]
samples_test = X_test.shape[0]
samples_val  = X_val.shape[0]

X_train_nn = X_train
X_test_nn = X_test


y_train_nn = y_train.reshape(samples_train,1)
y_test_nn = y_test.reshape(samples_test,1)
y_val = y_val.reshape(samples_val,1)

X_val = X_val.as_matrix()
X_train_nn=X_train_nn.as_matrix()
X_test_nn=X_test_nn.as_matrix()

In [ ]:
def train_clf(clf,X_train, X_test, y_train, y_test):
    model = clf.fit(X_train, y_train)
    print("Baseline")
    print("train error",mean_absolute_error(y_train, model.predict(X_train)))
    print("test error",mean_absolute_error(y_test, model.predict(X_test)))
    return 
train_clf(DummyRegressor(), X_train, X_test, y_train,y_test)

In [ ]:
def model_2_layers(h_layer_1_nodes, h_layer_2_nodes, A1, A2, l2):
    model = Sequential()
    model.add(Dense(h_layer_1_nodes, input_dim=X_train.shape[1], activation= A1,kernel_regularizer=regularizers.l2(l2)))
    model.add(Dense(h_layer_2_nodes, activation=A2 ,kernel_regularizer=regularizers.l2(l2)))
    model.add(Dense(1))
    return model

In [ ]:
model = model_2_layers(2, 2, "relu","relu", 0.05)
model.compile(loss='mean_absolute_error', optimizer='adam')
train_errs = []
test_errs = []
epoch = 200
counter = 0
best_err = 1000000000
last_epoch = epoch
for ep in range(1,epoch):
    model.fit(X_train_nn, y_train_nn, epochs=1, batch_size=32, shuffle=True,verbose=0)
    test_predictions = model.predict(X_test_nn, batch_size=32, verbose=0)
    train_predictions = model.predict(X_train_nn, batch_size=32, verbose=0)
    test_err = mean_absolute_error(y_test_nn, test_predictions)
    train_err = mean_absolute_error(y_train_nn, train_predictions)
    train_errs.append(train_err)
    test_errs.append(test_err)
    if best_err>test_err:
        model.save_weights("weights_task_2.h5")
        best_err = test_err
        counter = 0
    else:
        counter += 1
    if counter == 20:
        last_epoch = (ep+1)
        break


print(mean_absolute_error(y_train, train_predictions))
print(mean_absolute_error(y_test, test_predictions))
fig, ax = plt.subplots()
ax.plot(range(1,last_epoch), train_errs, "b--", label="train")
ax.plot(range(1,last_epoch), test_errs, "r--", label="valid")
ax.set(xlabel='epochs', ylabel='MAD',
       title='Mean Absolute error(epochs)')

ax.legend()
ax.grid()

Model error estemate

In [ ]:
output_flow_model = model_2_layers(2, 2, "relu","relu", 0.05)
output_flow_model.load_weights("weights_task_2.h5")
predictions = output_flow_model.predict(X_val, batch_size=32, verbose=0)
err = mean_absolute_error(y_val, predictions)
print(err)

In [ ]:
df = pd.read_pickle("df_task_2_sub")
df.drop(columns=["POS_CAT_before","POS_CAT_during"], inplace=True)

X_full_task_2 = df.as_matrix()
df = pd.read_pickle("df_t1_for_t2")
X_full_task_1 = df.as_matrix()

Just for visualization 

In [ ]:

model = Sequential()
model.add(Dense(2, input_dim=X_train.shape[1], activation= "relu",kernel_regularizer=regularizers.l2(0.05)))
model.add(Dense(2, activation="relu" ,kernel_regularizer=regularizers.l2(0.05)))
model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='adam')
model.summary()

Model: P(A)*P(B|A) where:                                                          
   P(A) - Probability custumer will countine to use the service                        
    
   P(B|A) - output via posterminal if the user countinue to use the service

In [ ]:
def model_3_layers(h_layer_1_nodes, h_layer_2_nodes, h_layer_3_nodes, A1, A2, A3, l2):
    model = Sequential()
    model.add(Dense(h_layer_1_nodes, input_dim=35, activation= A1,kernel_regularizer=regularizers.l2(l2)))
    model.add(Dense(h_layer_2_nodes, activation=A2 ,kernel_regularizer=regularizers.l2(l2)))
    model.add(Dense(h_layer_3_nodes, activation=A3,kernel_regularizer=regularizers.l2(l2)))
    model.add(Dense(2, activation='softmax'))
    return model

probability_model = model_3_layers(14, 34, 7, "relu","relu", "tanh", 0.0014492103248979129)
probability_model.load_weights("weights.h5")
output_flow_model = model_2_layers(2, 2, "relu","relu", 0.05)
output_flow_model.load_weights("weights_task_2.h5")

probs = probability_model.predict(X_full_task_1)[:,1]
outputs = output_flow_model.predict(X_full_task_2)
final_pos_flow = probs*outputs
